In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import cv2
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import keras
from keras.models import Sequential, Model,load_model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping,ModelCheckpoint
from google.colab.patches import cv2_imshow
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from keras.preprocessing import image
from keras.initializers import glorot_uniform

In [ ]:
# train -> train0+train1+train2
train_path="/content/gdrive/MyDrive/TRAIN_UP"
# test -> test0+test1+test2
test_path="/content/gdrive/MyDrive/TEST_UP"
class_names=os.listdir(train_path)
class_names_test=os.listdir(test_path)

In [ ]:
train_datagen = ImageDataGenerator(zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15)
test_datagen = ImageDataGenerator(zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15)

In [ ]:
train_generator = train_datagen.flow_from_directory("/content/gdrive/My Drive/TRAIN_UP")
test_generator = test_datagen.flow_from_directory("/content/gdrive/My Drive/TEST_UP")

Found 1392 images belonging to 3 classes.
Found 602 images belonging to 3 classes.


In [ ]:
def identity_block(X, f, filters, stage, block):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters

    X_shortcut = X
   
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X

In [ ]:
def convolutional_block(X, f, filters, stage, block, s=2):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [ ]:
def ResNet50(input_shape=(224, 224, 3)):

    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)

    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')


    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

In [ ]:
base_model = ResNet50(input_shape=(224, 224, 3))

In [ ]:
base_model.load_weights("/content/gdrive/My Drive/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5")

In [ ]:
headModel = base_model.output
headModel = Flatten()(headModel)
headModel= Dense(256, activation='relu', name='fc1',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel= Dense(128, activation='relu', name='fc2',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel = Dense( 3,activation='sigmoid', name='fc3',kernel_initializer=glorot_uniform(seed=0))(headModel)

In [ ]:
model = Model(inputs=base_model.input, outputs=headModel)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1 (Conv2D)                 (None, 112, 112, 64  9472        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                              

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
for layer in model.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7fbe376f2a00> False
<keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x7fbe376de340> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7fbe376de940> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7fbe376dee50> False
<keras.layers.core.activation.Activation object at 0x7fbe376dea00> False
<keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7fbe375f1d90> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7fbe375a8b20> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7fbe375b36a0> False
<keras.layers.core.activation.Activation object at 0x7fbe375a8bb0> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7fbe375b94c0> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7fbe375c0dc0> False
<keras.layers.core.activation.Activation object at 0x7fbe375b9d30> False
<keras.layers.convolutional

In [ ]:
es=EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=20)

In [ ]:
mc = ModelCheckpoint('/content/gdrive/My Drive/resnet_model.h5', monitor='val_accuracy')

In [ ]:
from keras.optimizers import Adam
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.0001, epsilon=1e-7, amsgrad=False)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Training The Model


In [ ]:
H = model.fit_generator(train_generator,validation_data =test_generator, epochs=50,verbose=1,callbacks=[mc,es])

<ipython-input-20-b70988c66097>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  H = model.fit_generator(train_generator,validation_data =test_generator, epochs=50,verbose=1,callbacks=[mc,es])


Epoch 1/50
44/44 [==============================] - 1314s 30s/step - loss: 1.7821 - accuracy: 0.6746 - val_loss: 0.5899 - val_accuracy: 0.7907
Epoch 2/50
44/44 [==============================] - 860s 20s/step - loss: 0.6491 - accuracy: 0.7802 - val_loss: 0.5366 - val_accuracy: 0.7973
Epoch 3/50
44/44 [==============================] - 931s 21s/step - loss: 0.5206 - accuracy: 0.7996 - val_loss: 0.4526 - val_accuracy: 0.8272
Epoch 4/50
44/44 [==============================] - 925s 21s/step - loss: 0.3771 - accuracy: 0.8499 - val_loss: 0.5208 - val_accuracy: 0.7857
Epoch 5/50
44/44 [==============================] - 876s 20s/step - loss: 0.4166 - accuracy: 0.8226 - val_loss: 0.5953 - val_accuracy: 0.7907
Epoch 6/50
44/44 [==============================] - 872s 20s/step - loss: 0.3741 - accuracy: 0.8477 - val_loss: 0.5825 - val_accuracy: 0.8023
Epoch 7/50
44/44 [==============================] - 944s 21s/step - loss: 0.3291 - accuracy: 0.8614 - val_loss: 0.4525 - val_accuracy: 0.8289
Epoch

In [ ]:
model.evaluate_generator(test_generator)

In [ ]:
model_json = model.to_json()
with open("/content/gdrive/My Drive/model.json","w") as json_file:
  json_file.write(model_json)

In [ ]:
from keras.models import model_from_json

In [ ]:
def predict_(image_path):
  #Load the Model from Json File
  json_file = open('/content/gdrive/My Drive/model.json', 'r')
  model_json_c = json_file.read()
  json_file.close()
  model_c = model_from_json(model_json_c)
  #Load the weights
  model_c.load_weights("/content/gdrive/My Drive/resenet_model.h5")
  #Compile the model
  opt = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999)
  model_c.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
  #load the image you want to classify
  image = cv2.imread(image_path)
  image = cv2.resize(image, (224,224))
  cv2_imshow(image)
  #predict the image
  preds = model_c.predict(np.expand_dims(image, axis=0))[0]
  prediction = max(preds)
  print(preds)

  if prediction==preds[0]:
      print("Predicted Label: 0")
  elif prediction==preds[1]:
      print("Predicted Label: 1")
  else:
      print("Predicted Label: 2")


In [ ]:
predict_("/content/gdrive/MyDrive/train.zip/train/28897_left.jpeg")

In [ ]:
predict_("/content/gdrive/MyDrive/train.zip/train/28780_right.jpeg")

In [ ]:
import pandas as pd
hist_df = pd.DataFrame(H.history) 

# save to json:  
hist_json_file = 'history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

# or save to csv: 
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
hist_df

In [ ]:
import matplotlib.pyplot as plt
loss_train = hist_df['loss']
loss_val = hist_df['val_loss']
epochs = range(30)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
loss_train = hist_df['accuracy']
loss_val = hist_df['val_accuracy']
epochs = range(30)
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='Validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
true_value = []
predicted_value = []
plt.figure(figsize=(10,10))
import os
from os import listdir
# get the path or directory
testImages = []
folder_dir = "/content/gdrive/MyDrive/TEST_UP/Test_img_0"
for i in os.listdir(folder_dir):
    # check if the image ends with png or jpg or jpeg
    if (i.endswith(".png") or i.endswith(".jpg")\
        or i.endswith(".jpeg")):
        testImages.append(folder_dir+'/'+i)
        true_value.append(0)
folder_dir = "/content/gdrive/MyDrive/TEST_UP/Test_img_1"
for i in os.listdir(folder_dir):
    # check if the image ends with png or jpg or jpeg
    if (i.endswith(".png") or i.endswith(".jpg")\
        or i.endswith(".jpeg")):
        testImages.append(folder_dir+'/'+i)
        true_value.append(1)
folder_dir = "/content/gdrive/MyDrive/TEST_UP/Test_img_2"
for i in os.listdir(folder_dir):
    # check if the image ends with png or jpg or jpeg
    if (i.endswith(".png") or i.endswith(".jpg")\
        or i.endswith(".jpeg")):
        testImages.append(folder_dir+'/'+i)
        true_value.append(2)


def predict_(image_path):
  #Load the Model from Json File
  json_file = open('/content/gdrive/My Drive/model.json', 'r')
  model_json_c = json_file.read()
  json_file.close()
  model_c = model_from_json(model_json_c)
  #Load the weights
  model_c.load_weights("/content/gdrive/My Drive/best_model.h5")
  #Compile the model
  opt = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999)
  model_c.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
  #load the image you want to classify
  image = cv2.imread(image_path)
  image = cv2.resize(image, (224,224))
  #cv2_imshow(image)
  #predict the image
  preds = model_c.predict(np.expand_dims(image, axis=0))[0]
  prediction = max(preds)
  #print(preds)

  if prediction==preds[0]:
      #print("Predicted Label: 0")
      predicted_value.append(0)
  elif prediction==preds[1]:
      #print("Predicted Label: 1")
      predicted_value.append(1)
  else:
      #print("Predicted Label: 2")
      predicted_value.append(2)


for image in testImages:
  print(image)
  predict_(image)





In [ ]:
li = []
for i in range(len(predicted_value)):
  li.append(i+1)
for i in range(len(predicted_value)):
  li[i] = li[i]+(predicted_value[i]-true_value[i])*10
Li = []
for i in range(i+1):
  Li.append(i+1)

plt.figure(figsize=(20,20))
plt.scatter(Li, li, c ="red",
            linewidths =1,
            marker ="*",
            edgecolor ="green",
            s = 200)
plt.scatter(Li, Li, c ="blue",
            linewidths =0.11,
            marker =".",
            edgecolor ="blue",
            s = 20)
plt.show()
